In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
import sys
import json
import urllib.request
import pandas as pd
import numpy as np
from model import *

Read words you want to scrape

In [2]:
urls = []
with open("words.txt", "r") as f:
    data = f.read().split("\n")
    
    for word in data:
        urls.append(f"http://5.vndic.net/index.php?word={word}&dict=en_vi")
print(urls[0:10])

['http://5.vndic.net/index.php?word=a&dict=en_vi', 'http://5.vndic.net/index.php?word=abandon&dict=en_vi', 'http://5.vndic.net/index.php?word=ability&dict=en_vi', 'http://5.vndic.net/index.php?word=able&dict=en_vi', 'http://5.vndic.net/index.php?word=abortion&dict=en_vi', 'http://5.vndic.net/index.php?word=about&dict=en_vi', 'http://5.vndic.net/index.php?word=above&dict=en_vi', 'http://5.vndic.net/index.php?word=abroad&dict=en_vi', 'http://5.vndic.net/index.php?word=absence&dict=en_vi', 'http://5.vndic.net/index.php?word=absolute&dict=en_vi']


Support function

In [3]:
def save(before, after):
    urllib.request.urlretrieve(before, after)

def trimAudio(audio):
    return re.findall(r"mp3.php\?id=\w+\&dir=\d+&lang=en\&", audio)[0]

In [4]:
# block = None
#     meaning = None
#     # for tr in trs:
#     #     print(tr)
#     for i, tr in enumerate(trs):
            
#         font = tr.find_all("font")
#         content = tr.getText()
        
#         if font == []:
#             if meaning:
#                 block.addMeaning(meaning)
#             meaning = Meaning(content)
#         else :
#             if font[0]["color"] == "#0033CC":
#                 if block:
#                     vocab.addTranslation(block)
#                 block = Block(content)
#             elif font[0]["color"] == "#CC3300":
#                 try:
#                     meaning.addExample(Example(
#                         content,
#                         trs[i + 1].getText()
#                     ))
#                 except:
#                     pass
#             elif font[0]["color"] == "#74726D":
#                 continue
#     if meaning:
#         block.addMeaning(meaning)
#     if block:
#         vocab.addTranslation(block)

Build scraping function

Strip() left and right of string

Remove "Xem" error in string

Remove duplicate in string


In [13]:

vocabularies = []
errorURL = []
def CRAWL(url):
    # Get from url
    try:
        r = requests.get(url)
    except:
        errorURL.append(url)
        return "ERROR in requesting!"
    
    
    soup = bs(r.text)
    try:
        word = soup.select(".thisword")[0].getText()
    except:
        return "ERROR in scraping word"
    
    # Scrape image
    try:
        img = str(soup.find_all("itemprop")[0].find_all("img")[0]["src"])
        if not img or word not in img:
            image_api = ""
            image = ""
        else :
            image_api = "http://5.vndic.net/" + img
            image = f"./Image/{word}.png"
            save(image_api, image)
    except:
        image_api = ""
        image = ""
    
    
    #Scrape audio
    try:
        ado = str(trimAudio(soup.find_all(style ="cursor:pointer;")[0]["onclick"]))
        if not ado or word not in ado:
            audio_api = ""
            audio = ""
        else :
            audio_api = "http://5.vndic.net/" + ado
            audio = f"./MP3/{word}.mp3"
            save(audio_api, audio)
    except:
        audio_api = ""
        audio = ""
    
    
    try:
        trs = soup.select(".maincontent")[0].find_all("tr")    
    except:
        trs = []
    
    if trs == []:
       
        return "ERROR in finding translation"
    
    spelling = trs[0].getText().strip()

    vocab = Vocabulary(word, image_api, image, audio_api, audio, spelling)

    block = None
    meaning = None
    # for tr in trs:
    #     print(tr)
    
    for i, tr in enumerate(trs):
        # print(tr)
        img = tr.find_all("img")
        src = ""
        if img:
            src = img[0]["src"]
            # print(src)
            if src == "img/dict/809C2811.png":
                break
        
        
        content = tr.getText().strip()
        
        if src == "img/dict/CB1FF077.png":
            if block and meaning:
                block.addMeaning(meaning)
            meaning = Meaning(content)
        
        elif src == "img/dict/46E762FB.png":
            if block and meaning:
                block.addMeaning(meaning)
                meaning = None
            if block and block.meanings:
                vocab.addTranslation(block)
            block = Block(content)
        elif src == "img/dict/9F47DE07.png":
            if tr.find_all("font")[0]["color"] == "#74726D":
                continue
            try:
                meaning.addExample(Example(
                    content,
                    trs[i + 1].getText().strip()
                ))
            except:
                pass
            
        
            
    if block and meaning:
        block.addMeaning(meaning)
    if block and block.meanings:
        vocab.addTranslation(block)
    

    vocabularies.append(vocab.toObj())
    # print(json.dumps(vocab.__dict__))

    # for block in vocab.translation:
    #     print(block)
    # print(word)
    
    return "SUCCESSFULL!"



Scraping vocabulary

In [16]:
success = 0
for i, url in enumerate(urls):
    res = CRAWL(url)
    
    if res == "SUCCESSFULL!":
        success += 1
    print(i, res)   

print("Rate successful : {}".format(success/3000)) 
        
with open('vocabulary.json', 'w', encoding='utf-16') as f:
    json.dump(vocabularies, f, ensure_ascii=False, indent=4)
# print(vocabularies)

Rate successful : 0.9906666666666667


Testing words after scraping

In [15]:
with open("vocabulary.json", encoding= 'utf-16') as f:
    data = json.load(f)
    
    
df = pd.DataFrame(data)
df.to_csv("vocabulary.csv")

df.head()

,word,image_api,image,audio_api,audio,spelling,translation1,translation2,translation3,translation4,translation5,translation6
0,abandon,http://5.vndic.net/images/dict/a/abandon.gif,./Image/abandon.png,http://5.vndic.net/mp3.php?id=abandon&dir=957&...,./MP3/abandon.mp3,[ə'bændən],"{'type': 'ngoại động từ', 'meanings': [{'meani...","{'type': 'danh từ', 'meanings': [{'meaning': '...",NaN,NaN,NaN,NaN
1,ability,http://5.vndic.net/images/dict/a/ability.gif,./Image/ability.png,http://5.vndic.net/mp3.php?id=ability&dir=137&...,./MP3/ability.mp3,[ə'biliti],"{'type': 'danh từ', 'meanings': [{'meaning': '...",NaN,NaN,NaN,NaN,NaN
2,able,http://5.vndic.net/images/dict/a/able.gif,./Image/able.png,http://5.vndic.net/mp3.php?id=able&dir=287&lan...,./MP3/able.mp3,['eibl],"{'type': 'tính từ', 'meanings': [{'meaning': '...","{'type': 'hậu tố', 'meanings': [{'meaning': 'c...",NaN,NaN,NaN,NaN
3,abortion,http://5.vndic.net/images/dict/a/abortion.gif,./Image/abortion.png,http://5.vndic.net/mp3.php?id=abortion&dir=810...,./MP3/abortion.mp3,[ə'bɔ:∫n],"{'type': 'danh từ', 'meanings': [{'meaning': '...",NaN,NaN,NaN,NaN,NaN
4,about/on one's person,,,,,thành ngữ person,NaN,NaN,NaN,NaN,NaN,NaN
